In [1]:
import os
import sys
os.chdir('../')
sys.path.append('../')

import torch
from utils.utils import evaluate,train,prepare
from models.FIM import FIMModel

## setting up the *FIM* model

### define paths and hyperparameters, load data

all these hyper parameters are fixed according to the paper [\[29\] Fine-grained Interest Matching for Neural News Recommendation](https://www.aclweb.org/anthology/2020.acl-main.77.pdf)

- *mode*: data to read (*demo*/*small*/*large*)

- *batch_size*: size of each minibatch

- *title_size*: max word capacity of title

- *his_size*: max record capacity of click history

- *npratio*: number of negtive sampling

- *dilation_level*: levels of diferrent dilation rate

- *kernel_size*: size of 1dCNN kernel

- *filter_num*: number of kernels in 1D CNN, which is also embedding dimension of news/user

- *embedding_dim*: word embedding dimension

- *metrics*: metrics to be used in evaluating

- *gpu*: gpu ID if available

- *attrs*: the columns used to construct vocab

### Note 

Although I list *dilation_level* and *kernel_size*, some properties related to both variables are fixed in the FIM model because I don't have time to well-design my model to make it adapt to these variables dynamically. 

In [2]:
hparams = {
    'scale':'demo',
    'name':'fim',
    'batch_size':100,
    'title_size':20,
    'his_size':50,
    'kernel_size':3,
    'npratio':4,
    'dropout_p':0.2,
    'dilation_level':3,
    'filter_num':150,
    'embedding_dim':300,
    'metrics':'group_auc,ndcg@5,ndcg@10,mean_mrr',
    'device':'cuda:0',
    'attrs': ['title'],
    'epochs':10,
    'k':None
    'save_step':None,
    'save_each_epoch':False,
    'train_embedding':False,
}

In [3]:
device = torch.device(hparams['device'])

vocab, loader_train, loader_test, loader_validate = prepare(hparams, validate=True)

In [4]:
fimModel = FIMModel(vocab=vocab,hparams=hparams).to(device)

### train the model

In [5]:
train(fimModel, hparams, loader_train, loader_test, loader_validate, save=False)

training...
epoch 0 , step 10 , loss: 1.7415: : 20it [00:05,  3.51it/s]
epoch 1 , step 10 , loss: 1.5952: : 20it [00:05,  3.76it/s]
epoch 2 , step 10 , loss: 1.4876: : 20it [00:05,  3.70it/s]
epoch 3 , step 10 , loss: 1.4288: : 20it [00:05,  3.75it/s]
epoch 4 , step 10 , loss: 1.3672: : 20it [00:05,  3.78it/s]
epoch 5 , step 10 , loss: 1.3079: : 20it [00:05,  3.80it/s]
epoch 6 , step 10 , loss: 1.2801: : 20it [00:05,  3.77it/s]
epoch 7 , step 10 , loss: 1.2189: : 20it [00:05,  3.78it/s]
epoch 8 , step 10 , loss: 1.1898: : 20it [00:05,  3.75it/s]
epoch 9 , step 10 , loss: 1.1284: : 20it [00:05,  3.77it/s]
0it [00:00, ?it/s]save success!
testing...
181it [00:11, 15.45it/s]
0it [00:00, ?it/s]evaluation results:{'group_auc': 0.562, 'ndcg@5': 0.2749, 'ndcg@10': 0.3394, 'mean_mrr': 0.2647}
validating...
753it [00:45, 16.39it/s]
evaluation results:{'group_auc': 0.8108, 'ndcg@5': 0.5193, 'ndcg@10': 0.5678, 'mean_mrr': 0.4691}
